# Импорт библиотек

In [1]:
import numpy as np
from gzip import open # NB: overrides standard open()
import pickle as pkl
import pandas as pd
import warnings 
from glob import glob
import os
import math
import functools

### Отключим предупреждения Anaconda

In [2]:
warnings.simplefilter('ignore')

# Загрузим данные

In [3]:
Xdata_numpy = pkl.load(open('C:/Users/admin/Desktop/PLS_continue_2020/X.pkl.gz', 'rb'))
Ydata = pkl.load(open('C:/Users/admin/Desktop/PLS_continue_2020/y.pkl.gz', 'rb'))

# N-PLS1

## Выберем столбец у с которым будем работать

In [4]:
m=0

# Приближение к правильному алгоритму

In [104]:
from sklearn.model_selection import train_test_split

In [219]:
X_train, X_test, y_train, y_test = train_test_split(
    Xdata_numpy["X"], Ydata.iloc[:,m], test_size=0.2857 
    )

In [220]:
X_train.shape[0]

25

In [279]:
x=X_train

In [280]:
y=y_train

In [281]:
y=np.array(y)

In [282]:
x=np.array(x)

In [264]:
yo=y

In [232]:
# a-numpy array;  b-pandas array
def error(a,b):
    s=0
    for i in range(0,len(a)):
        s+=((a[i]-b.iloc[i]))**2
    return s

In [283]:
def function_1(x,y,fn):
    Tt=[]
    mass=np.zeros([y.shape[0]])
    output=[]
    for f in range(0,fn):
        z=np.zeros([x.shape[1],x.shape[2]])
        for i in range(0,x.shape[2]):
            for j in range(0,x.shape[1]):
                zsum=0
                for k in range(0,x.shape[0]):
                    zsum= zsum+ x[k,j,i]*y[k]
                z[j,i]=zsum
        Wk, S, WI = np.linalg.svd(z)
        w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
        w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
    
        t=[]
        for h in range(0,x.shape[0]):
            t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
        t=np.array(t).reshape(x.shape[0],1)
        Tt+=[t]
        T=np.array(Tt).reshape(x.shape[0],f+1)
        bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
    
        for g in range(0,x.shape[0]):
            x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
        y=y-(np.dot(T,bf))
        mass+=np.dot(T,bf)
        output+=[error(mass,y_train)]
    return output

#### Ниже изначальная, но, пока ещё нужная для работы версия function_1

###### Массив mass собирает разложение обратно, и именно он важен для определения числа комронент

In [270]:
Tt=[]
mass=np.zeros([y.shape[0]])
for f in range(0,55):
    z=np.zeros([x.shape[1],x.shape[2]])
    for i in range(0,x.shape[2]):
        for j in range(0,x.shape[1]):
            zsum=0
            for k in range(0,x.shape[0]):
                zsum= zsum+ x[k,j,i]*y[k]
            z[j,i]=zsum
    Wk, S, WI = np.linalg.svd(z)
    w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
    w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
    
    t=[]
    for h in range(0,x.shape[0]):
        t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
    t=np.array(t).reshape(x.shape[0],1)
    Tt+=[t]
    T=np.array(Tt).reshape(x.shape[0],f+1)
    bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
    for g in range(0,x.shape[0]):
        x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
    y=y-(np.dot(T,bf))
    mass+=np.dot(T,bf)
    print(error(mass,y_train))

6295.280934294746
5025.960053166011
4537.110878008037
3910.682162173213
3508.8721869572855
3110.8690063698773
2178.2835802681734
1899.1215010476215
1160.0942926802636
826.8799914544466
664.9274098898748
475.8662917913025
343.3229431335154
183.84361608155476
29.057468644372353
16.795243483399545
9.225309332501615
8.463611118343504
6.962010905198145
4.381841364865522
0.28480648440565753
0.024950819146111212
0.000154575371231868
6.529091562035779e-07
1.5865161616894563e-09
1.899487740971464e-16
7.736229698890818e-17
1.210558321571224e-16
1.6009409235644413e-16
1.2612494789381252e-15
3.839935857203881e-17
4.1651715241658746e-17
6.23224806844377e-15
4.5243862857533e-13
4.22165629797906e-12
1.057150294646872e-12
1.1392093657824908e-12
2.3205072089838744e-11
3.8599517624417894e-11
6.218014854723555e-10
1.1391395220420335e-09
6.908513219144009e-06
9.014284003811633e-09
2.096458782321592e-06
0.00012051987536252695
0.0005695468530131893
0.0007787708690048402
2.6685555435564994
2.3123077137238193

# Попробую оформить всё в виде полноценного класса

In [ ]:
class N_PLS1():
    def error(a,b):
        s=0
        for i in range(0,len(a)):
            s+=((a[i]-b.iloc[i]))**2
        return s
    def function_1(x,y,fn):
        Tt=[]
        mass=np.zeros([y.shape[0]])
        output=[]
        for f in range(0,fn):
            z=np.zeros([x.shape[1],x.shape[2]])
            for i in range(0,x.shape[2]):
                for j in range(0,x.shape[1]):
                    zsum=0
                    for k in range(0,x.shape[0]):
                        zsum= zsum+ x[k,j,i]*y[k]
                    z[j,i]=zsum
            Wk, S, WI = np.linalg.svd(z)
            w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
            w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
            t=[]
            for h in range(0,x.shape[0]):
                t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
            t=np.array(t).reshape(x.shape[0],1)
            Tt+=[t]
            T=np.array(Tt).reshape(x.shape[0],f+1)
            bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
            for g in range(0,x.shape[0]):
                x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
            y=y-(np.dot(T,bf))
            mass+=np.dot(T,bf)
            output+=[error(mass,y_train)]
        return output
    def fit(x,y,fn):
        
            
        return y

In [179]:
y

array([34.19308759, 35.72443085, 46.32211944, 35.69579348, 47.44184036,
       33.34204937, 42.44462856, 39.9018207 , 44.39973652, 54.64066407,
       53.71596802, 44.04933373, 36.11255278, 34.24246421, 36.22622795,
       26.1974747 , 41.0398005 , 31.41066637, 34.07047331, 42.14770231,
       45.44679116, 21.41118209, 44.54683093, 35.35460545, 36.47993911])

In [54]:
print(y.shape)

()


In [284]:
function_1(x,y,25)

[6295.280934294746,
 5025.960053166011,
 4537.110878008037,
 3910.682162173213,
 3508.8721869572855,
 3110.8690063698773,
 2178.2835802681734,
 1899.1215010476215,
 1160.0942926802636,
 826.8799914544466,
 664.9274098898748,
 475.8662917913025,
 343.3229431335154,
 183.84361608155476,
 29.057468644372353,
 16.795243483399545,
 9.225309332501615,
 8.463611118343504,
 6.962010905198145,
 4.381841364865522,
 0.28480648440565753,
 0.024950819146111212,
 0.000154575371231868,
 6.529091562035779e-07,
 1.5865161616894563e-09]